In [1]:
import pandas as pd
import torch
import GPUtil
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt 
import numpy as np 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


### Cargar base de datos de entrenamiento 

In [2]:
# Cargar conjunto de entraniento 
trainingData = pd.read_csv(r'S:\Proyecto Epilepsia\TTV\Training.csv')  # Base da datos completa col = 'Sample', 'Target'
X_train = trainingData['Sample']   # Asignar columna de samples a X
y_train = trainingData['Target']   # Asignar columna de target a y 

# Cargar conjunto de testing
testingData = pd.read_csv(r'S:\Proyecto Epilepsia\TTV\Testing.csv')  # Base da datos completa col = 'Sample', 'Target'
X_testing = testingData['Sample']   # Asignar columna de samples a X
y_testing = testingData['Target']   # Asignar columna de target a y 

# Cargar conjunto de validation
valData = pd.read_csv(r'S:\Proyecto Epilepsia\TTV\Validation.csv')  # Base da datos completa col = 'Sample', 'Target'
X_val = valData['Sample']   # Asignar columna de samples a X
y_val = valData['Target']   # Asignar columna de target a y 

In [3]:
# Convertir a arrelos training 
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

# Convertir a arrelos testing
X_testing = X_testing.to_numpy()
y_testing = y_testing.to_numpy()

# Convertir a arrelos validation 
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()

### Definir transformaciones para base de datos 

In [4]:
# Media y desviación estandar del conjunto de entrenamiento
genMean = [0.2487, 0.2487, 0.2487]
genSD = [0.2377, 0.2377, 0.2377]


In [5]:
# Definir transformacion para la imagen 
traingTans = transforms.Compose([ transforms.RandomHorizontalFlip(p=0.5), # Hace un flip en el sentido horizontal 
                                  transforms.ToTensor(), # toma los valores iniciales y los escala a valores entre 0-1
                                  transforms.Normalize(genMean, genSD)
                                  ],
                               ) 

testTrans = transforms.Compose([ transforms.ToTensor(), # toma los valores iniciales y los escala a valores entre 0-1
                                 transforms.Normalize(genMean, genSD)
                                  ],
                               ) 

In [6]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, targets, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Cargar la imagen y convertir a RGB
        target = self.targets[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, target
    

In [7]:
## Muestras de training
trainSample = CustomImageDataset(image_paths=X_train,targets=y_train,transform = traingTans)

# Muestras testing
testSample = CustomImageDataset(image_paths=X_testing,targets=y_testing,transform = testTrans)

# Muestras val 
valSample = CustomImageDataset(image_paths=X_val,targets=y_val,transform = testTrans)


In [8]:
# Preparar data loader
batch = 32

trainLoader = DataLoader(dataset=trainSample, batch_size=batch,shuffle=True)
testingLoader = DataLoader(dataset=testSample, batch_size=batch,shuffle=True)
valLoader = DataLoader(dataset=valSample, batch_size=batch,shuffle=True)

# DataLoader -> Facilita la carda de datos para entrenamiento y evaluación de modelos
# Carga los datos en mini-batches para tener fragmentos manejables y actualizar los pesos
# Genera aleatoridad en los datos para ayudar a la generalización de los datos
# Proporiona interfaz iterable

### Mostrar imagenes de data iter 


In [ ]:
dataiter = iter(trainLoader)

img,lb = next(dataiter)

In [ ]:
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.2486, 0.2486, 0.2486])
        std = np.array([0.2377, 0.2377, 0.2377])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

In [ ]:
print(img[0].shape)
imgTrans = img[0].numpy().transpose((1, 2, 0)) # transpone las dimensiones del arrelo en este caso incialmente se tiene (3,256,256) -> (256,256,3)
print(imgTrans.shape)
plt.imshow(imgTrans)

In [ ]:
imshow(img[0])
imshow(img[0],normalize=False)

### Selección de dispositivos de entrenamiento

In [9]:
device = torch.cuda.is_available()

if not device:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [10]:
# Tipo de GPU donde se está entrenando la red
gpus = GPUtil.getGPUs()
if gpus:
    for gpu in gpus:
        print(f"ID: {gpu.id}, Name: {gpu.name}, Load: {gpu.load*100}%, Memory Free: {gpu.memoryFree}MB, Memory Used: {gpu.memoryUsed}MB, Memory Total: {gpu.memoryTotal}MB, Temperature: {gpu.temperature} °C")
else:
    print("No GPUs available")

ID: 0, Name: NVIDIA GeForce RTX 4070 Laptop GPU, Load: 15.0%, Memory Free: 7827.0MB, Memory Used: 122.0MB, Memory Total: 8188.0MB, Temperature: 47.0 °C


## Estructura CCN (3 Capas)

In [11]:
# define the CNN architecture
import torch.nn as nn
import torch.nn.functional as F

# img - entrada -> 256
class CNNet(nn.Module):
    def __init__(self):
        super(CNNet, self).__init__()
    
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(10)        
        
        self.conv2 = nn.Conv2d(in_channels=10,out_channels=20,kernel_size=3,padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(20)
        
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=30,kernel_size=3, padding=1, stride=1)
        self.bn3 = nn.BatchNorm2d(30)
        
        
        self.pool = nn.MaxPool2d(2,2)
        
        # Multilayer
        self.fc1 = nn.Linear(30*32*32,5000)  # 16*16 es por la redución debido a la capa de pooling 
        self.fc2 = nn.Linear(5000,500)
        self.fc3 = nn.Linear(500,2) 
        
        self.dropout = nn.Dropout(0.35)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(F.relu(x)) # debido a la capa de pooling se reduce la dim de 256*256 -> 128*128
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(F.relu(x)) # debido a la capa de pooling se reduce la dim de 128*128 -> 64*64
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(F.relu(x)) # debido a la capa de pooling se reduce la dim de 64*64 -> 32*32 
        
        x = x.view(-1, 30 * 32 * 32)
        # Capa de dropout
        x = self.dropout(x)
        # primer capa fully connected 
        x = F.relu(self.fc1(x))
        # Capa dropout 
        x = self.dropout(x)
        # Segunda capa fully connected 
        x = F.relu(self.fc2(x))
        # Capa dropout
        x = self.dropout(x)
        # Tercer capa fully connected 
        x = self.fc3(x) 
        
        return x

### Estructura de Red Neuronal (4 capas)

In [23]:
# define the CNN architecture
import torch.nn as nn
import torch.nn.functional as F

# img - entrada -> 256
class CNNet(nn.Module):
    def __init__(self):
        super(CNNet, self).__init__()
    
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(10)        
        
        self.conv2 = nn.Conv2d(in_channels=10,out_channels=20,kernel_size=3,padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(20)
        
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=30,kernel_size=3, padding=1, stride=1)
        self.bn3 = nn.BatchNorm2d(30)
        
        self.conv4 = nn.Conv2d(in_channels=30,out_channels=40,kernel_size=3, padding=1, stride=1)
        self.bn4 = nn.BatchNorm2d(40)
        
        self.pool = nn.MaxPool2d(2,2)
        
        # Multilayer
        self.fc1 = nn.Linear(40*16*16,5000)  # 16*16 es por la redución debido a la capa de pooling 
        self.fc2 = nn.Linear(5000,500)
        self.fc3 = nn.Linear(500,2) 
        
        self.dropout = nn.Dropout(0.35)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(F.relu(x)) # debido a la capa de pooling se reduce la dim de 256*256 -> 128*128
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(F.relu(x)) # debido a la capa de pooling se reduce la dim de 128*128 -> 64*64
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(F.relu(x)) # debido a la capa de pooling se reduce la dim de 64*64 -> 32*32 
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(F.relu(x))  # debido a la capa de pooling se reduce la dim de 32*32 -> 16*16 
        
        x = x.view(-1, 40 * 16 * 16)
        # Capa de dropout
        x = self.dropout(x)
        # primer capa fully connected 
        x = F.relu(self.fc1(x))
        # Capa dropout 
        x = self.dropout(x)
        # Segunda capa fully connected 
        x = F.relu(self.fc2(x))
        # Capa dropout
        x = self.dropout(x)
        # Tercer capa fully connected 
        x = self.fc3(x) 
        
        return x

## Proceso de entrenamiento y definición de red 

In [24]:
model = CNNet()
model

CNNet(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(30, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=10240, out_features=5000, bias=True)
  (fc2): Linear(in_features=5000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=2, bias=True)
  (dropout): Dropout(p=0.35, inplace=False)
)

In [25]:
# move tensors to GPU if CUDA is available
if device:
    model.cuda()

Criterio, optimizador y función de pérdida

In [26]:
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001)

Entrenamiento de red

In [27]:
# number of epochs to train the model
n_epochs = 100

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in trainLoader:
        # move tensors to GPU if CUDA is available
        if device:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valLoader:
        # move tensors to GPU if CUDA is available
        if device:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(trainLoader.sampler)
    valid_loss = valid_loss/len(valLoader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_CNN_SGD_100.pt')
        valid_loss_min = valid_loss

KeyboardInterrupt: 

## Prueba del modelo 

Modelo 3 capas 

In [28]:
model.load_state_dict(torch.load('model_CNN_EL.pt'))  # Cargar datos del modelo de 3 capas 

<All keys matched successfully>

In [29]:
# track test loss
# test_loss = 0.0
TP = 0.0
TN = 0.0
FP = 0.0
FN = 0.0
posFR = 1

model.eval()
# iterate over test data
for data, target in testingLoader:
    if device:
        data, target = data.cuda(), target.cuda()  # Enviar tensores a cuda
    
    output = model(data)  # Da la salida en valores calculados por la red 
    
    loss = criterion(output, target)  # Criterio de pérdida que emplea CrossEntropyLoss
     
    # test_loss += loss.item()*data.size(0)  # Sumatoria de perdidas 
    
    _, pred = torch.max(output, 1)   # Regresa el índice del arrego de salida con el valor más alto  para determinar la clase a la que pertenece 
    
    #Imprimir target y predicción     
    # print('Preiction: ', pred)
    # print('Target: ', target)
    
    ## Statics calculation
    TP += torch.sum((target == posFR) & (pred == posFR)).item()
    # print(TP)
    TN += torch.sum((target != posFR) & (pred != posFR)).item()
    # print(TN)
    FP += torch.sum((target != posFR) & (pred == posFR)).item()
    # print(FP)
    FN += torch.sum((target == posFR) & (pred != posFR)).item()
    # print(FN)

# test_loss = test_loss/len(testingLoader.dataset)
# print('Test Loss: {:.6f}\n'.format(test_loss))
    

## Calculo de mediads estadisticas 

In [30]:
print('TP: ',TP)
print('TN: ',TN)
print('FP: ',FP)
print('FN: ',FN)

TP:  346.0
TN:  8232.0
FP:  175.0
FN:  121.0


In [31]:
## Sensitividad:
Sen = (TP)/(TP+FN)*100
print(Sen)
Spc = (TN)/(TN+FP)*100
print(Spc)
Pres = (TP)/(TP+FP)*100
print(Pres)
Acc = (TP+TN)/(TP+FN+TN+FP)*100 
print(Acc)

74.08993576017131
97.91840133222316
66.41074856046065
96.66441289159341
